## Playground

In [1]:
pip install openai

In [2]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [3]:
import os

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [4]:
import requests

def get_weather_yiqiao(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']

Call model with functions defined – along with your system and user messages

In [13]:
from openai import OpenAI
import json

client = OpenAI()

tools = [{
    "type": "function",
    "name": "get_weather_yiqiao",
    "description": "Get current temperature for provided coordinates in celsius.",
    "parameters": {
        "type": "object",
        "properties": {
            "latitude": {"type": "number"},
            "longitude": {"type": "number"}
        },
        "required": ["latitude", "longitude"],
        "additionalProperties": False
    },
    "strict": True
}]

input_messages = [{"role": "user", "content": "What's the weather like in Paris today?"}]

response = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)

Model decides to call function(s) – model returns the name and input arguments.

In [14]:
response

Response(id='resp_68640c66dfb88199ab2673a73b55d5630ac7d35aec971f58', created_at=1751387238.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_cf28AO6olYogSmcL3DohMNty', name='get_weather_yiqiao', type='function_call', id='fc_68640c680fa88199b06b28f676e1402a0ac7d35aec971f58', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather_yiqiao', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for provided coordinates in celsius.')], top_p=1.0, background=False, max_output_tokens=None, previous_response_id=None, prompt=None, reasoning=Reasoning(effort=None, generate

Execute function code – parse the model's response and handle function calls.

In [15]:
tool_call = response.output[0]
args = json.loads(tool_call.arguments)
print(args)

result = get_weather_yiqiao(args["latitude"], args["longitude"])

{'latitude': 48.8566, 'longitude': 2.3522}


In [16]:
result

38.3

Supply model with results – so it can incorporate them into its final response.

In [17]:
tool_call

ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_cf28AO6olYogSmcL3DohMNty', name='get_weather_yiqiao', type='function_call', id='fc_68640c680fa88199b06b28f676e1402a0ac7d35aec971f58', status='completed')

In [18]:
input_messages

[{'role': 'user', 'content': "What's the weather like in Paris today?"}]

In [19]:
input_messages.append(tool_call)                      # append model's function call message
input_messages.append({                               # append result message
    "type": "function_call_output",
    "call_id": tool_call.call_id,
    "output": str(result)
})

response_2 = client.responses.create(
    model="gpt-4.1",
    input=input_messages,
    tools=tools,
)
print(response_2.output_text)

The weather in Paris today is quite hot, with a temperature of around 38.3°C. If you're planning to go out, make sure to stay hydrated and protect yourself from the sun!


In [ ]:
input_messages

[{'role': 'user', 'content': "What's the weather like in Paris today?"},
 ResponseFunctionToolCall(arguments='{"latitude":48.8566,"longitude":2.3522}', call_id='call_7AG9GZ8IYBJszi7w3RsU4qS5', name='get_weather_yiqiao', type='function_call', id='fc_685ecce6a7d0819b8437de28e8242f280d2d526619bec5b3', status='completed'),
 {'type': 'function_call_output',
  'call_id': 'call_7AG9GZ8IYBJszi7w3RsU4qS5',
  'output': '27.9'}]

In [ ]:
response_2

Response(id='resp_685ecd253cac819bb2cc061cda755cd60d2d526619bec5b3', created_at=1751043365.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_685ecd25c66c819bbc33aad68d15aeab0d2d526619bec5b3', content=[ResponseOutputText(annotations=[], text='Today in Paris, the weather is currently about 27.9°C. It’s quite warm, so you may want to dress light if you’re going out!', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather_yiqiao', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for provided coordinates in celsius.')], top_p=1.0, background=False, max_o

## Framework

In [50]:
# chat_manager.py
import os
import json
from typing import Callable, List, Dict, Optional, Any, Union

import requests
from pydantic import BaseModel
from openai import OpenAI


class ToolCallArguments(BaseModel):
    """Pydantic model to enforce arguments passed to tools."""
    latitude: float
    longitude: float


class ChatManager:
    """
    A manager class that handles conversation with an LLM, including tool use.
    """

    def __init__(
        self,
        api_key: str,
        system_prompt: Optional[str] = None,
        history: Optional[List[Dict[str, Any]]] = None,
        tool_functions: Optional[List[Callable]] = None,
        tool_descriptions: Optional[List[Dict[str, Any]]] = None,
    ) -> None:
        os.environ["OPENAI_API_KEY"] = api_key
        self.client = OpenAI()
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
        if history:
            self.messages.extend(history)
        self.tool_functions = {tool.__name__: tool for tool in (tool_functions or [])}
        self.tool_descriptions = tool_descriptions or []

    def _run_tool(self, name: str, arguments: Dict[str, Any]) -> Any:
        """
        Executes the tool function by name with the given arguments.
        """
        if name not in self.tool_functions:
            raise ValueError(f"Tool '{name}' is not registered.")
        validated_args = ToolCallArguments(**arguments)
        return self.tool_functions[name](**validated_args.model_dump())

    def converse(self, user_input: str) -> str:
        """
        Main conversational loop. Handles tool invocations if needed.
        """
        self.messages.append({"role": "user", "content": user_input})
        response = self.client.responses.create(
            model="gpt-4.1",
            input=self.messages,
            tools=self.tool_descriptions,
        )

        tool_call = response.output[0]

        # If the model made a tool call
        if tool_call.type == "function_call":
            arguments = json.loads(tool_call.arguments)
            print(f"[Tool Invoked] {tool_call.name} with arguments: {arguments}")
            output = self._run_tool(tool_call.name, arguments)
            self.messages.append(tool_call)
            self.messages.append({
                "type": "function_call_output",
                "call_id": tool_call.call_id,
                "output": str(output)
            })
            follow_up = self.client.responses.create(
                model="gpt-4.1",
                input=self.messages,
                tools=self.tool_descriptions,
            )
            reply = follow_up.output_text
            # print("\nBot:", reply)
            print("[Full Follow-up Output]\n", follow_up)
            return reply

        # Regular text response
        print("[Full Response Output for developers]\n", response)
        reply = response.output[0].content[0].text
        return reply


def get_weather_yiqiao(latitude: float, longitude: float) -> float:
    """
    Get the current temperature at the given coordinates using Open-Meteo API.

    Args:
        latitude (float): The latitude of the location.
        longitude (float): The longitude of the location.

    Returns:
        float: The current temperature in Celsius.
    """
    response = requests.get(
        f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m"
    )
    data = response.json()
    return data['current']['temperature_2m']


if __name__ == "__main__":
    # from google.colab import userdata
    # OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") # assume you already define it above

    tools = [{
        "type": "function",
        "name": "get_weather_yiqiao",
        "description": "Get current temperature for provided coordinates in Celsius.",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"],
            "additionalProperties": False
        },
        "strict": True
    }]

    manager = ChatManager(
        api_key=OPENAI_API_KEY,
        system_prompt="You are a helpful assistant.",
        tool_functions=[get_weather_yiqiao],
        tool_descriptions=tools
    )

    # Single turn conversation
    # result = manager.converse("What's the weather like in Paris today?")
    # print(result)

    # Multi turn conversation
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit" or user_input.lower() == 'quit':
            break

        result = manager.converse(user_input)
        print("Bot: ", result)


User: hi
[Full Response Output for developers]
 Response(id='resp_686447015be08198b7cdcab6b64f7f4f0678d2dd9b1495be', created_at=1751402241.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_68644701ef0081989671a6f2ea35654d0678d2dd9b1495be', content=[ResponseOutputText(annotations=[], text='Hello! How can I assist you today?', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_weather_yiqiao', parameters={'type': 'object', 'properties': {'latitude': {'type': 'number'}, 'longitude': {'type': 'number'}}, 'required': ['latitude', 'longitude'], 'additionalProperties': False}, strict=True, type='function', description='Get current temperature for provided coordinates in Celsius.')], top_p=1.0, background=False, max_output_tokens=None, previous_response_id=N